The following notebook provides my thoughts on the beta API provided via https://data-beta.usgs.gov/sgcn/ as of August 19, 2020. I provide some notes about what I'm seeing inline with code examples I used to exercise the API.

# General Principles
The following are some general assumptions and design principles that I believe apply across the board for these types of systems. They should be evident throughout my thinking about the SGCN case.

* This and others we've been working on for DRIP and USNVC are intended for immediate advertisement on the front page of whatever web site is the home location for these data and their functionality. They are intended for broad public audiences and many unanticipated downstream uses. As such, they need to fully abstract away from whatever underlying technology we are using in the moment to store and provide the data for use. This is to give us a layer of protection from future changes in that underlying infrastructure by ensuring that downstream applications are independent from changes we need to make.
* Subject matter experts in the particular data asset being distributed through an API like this need to be the ones to provide documentation on the API routes, input variables, and response data models. I had intended that this would be done directly via things like JSON Schema artifacts and even code contributions to something like a Flask REST app as a litmus test of this principle. This means that everything in the API needs to be fully understandable by those SMEs so that they can explain it in plain language for downstream users. Every time we introduce an esoteric term or something weird in a route that the SME can't figure out and explain for themselves, we've violated this principle.

In [7]:
import requests
from IPython.display import display

In [2]:
r_national_list = requests.get("https://data-beta.usgs.gov/sgcn/api/v1/nationalList?pagesize=10&offset=0&order_by=State&year=2015").json()

Incorporation of URL followable references like this are great. We should use them wherever we have a resolvable reference. But we need the software we are using for the API to always be fully aware of what domain the app is being deployed to. I see that this sgcn-beta domain works from the internal network, but it doesn't match the data-beta locale, obviously. I thought there was functionality built into Flask for handling this, if that's what we're still using.

In [8]:
display(r_national_list["thispage"])

'https://sgcn-beta.snafu.cr.usgs.gov/sgcn/api/v1/nationalList?offset=0&pagesize=10&year=2015&order_by=state'

I would prefer to get rid of all this stuff entirely from the visible API response. It is just confusing matters for any type of end user that we are wanting to support through this thing. It's fine that the simple query parameters get translated into this behind the scenes, but the layer of abstraction principle means that we never show it (unless we need to present it via some kind of separate audit response for transparency purposes with certain kinds of data).

In [9]:
display(r_national_list["q"])

"{'from': 0, 'size': 10, 'query': {'bool': {'should': [{'match': {'data.match_method': 'Exact Match'}}, {'match': {'data.match_method': 'Fuzzy Match'}}, {'match': {'data.match_method': 'Followed Accepted TSN'}}, {'match': {'data.match_method': 'Found multiple matches'}}], 'minimum_should_match': 1, 'filter': {'term': {'data.year': 2015}}}}, 'sort': [{'data.state.keyword': {'order': 'asc'}}]}"

Per my comment about SMEs providing documentation on the API, including on the response model itself, there's a problem in how these records have been structured. First, have to get into a secondary data structure within the array of records to get at actual data. That's annoying to some extent but could be useful if there was a logical separation of content for a given record. In this case, we've stuck a few pieces of metadata at the top of record (run_id, row_id, and uri), but there are also additional metadata in the data "document." It might be more reasonable as a general design pattern to do something like the following:

* Separate data and metadata into two structures (objects) of key/value pairs and make sure that metadata/documentation properties and data properties are in their appropriate spot
* Make return of metadata a configurable option in the request, perhaps defaulting to true as best practice but allowing applications to return only essential information for the specific requirement

Note: This would be a good area to engage with Abby on to make a distinction between what she would consider to be metadata vs. data in the case of SGCN data.

More complex and lengthy data structures than this (but maybe even in this case) should probably incorporate a "fields" concept into the API allowing configuration on the specific data/metadata fields returned. Even more complicated cases often use a fieldsets idea where logical groupings of fields can be called up for types of uses. As a general rule, I would build this functionality into every data distribution API and then decide how it should be employed based on stakeholder input.

We also need to make sure that identifier is meaningful, explainable, and resolvable (wherever possible). I have no idea what run_id is here and whether or not I should care about it. It's probably important, and there should probably be a resolver for it. I find it easiest in REST APIs with JSON responses to carry namespace designations forward into the data values themselves to make things like this URL resolvable, but an alternative that can be useful is the JSON-LD approach with referral to namespace designations to obtain a resolution path.

In [13]:
print(r_national_list["records"][0].keys())
print(r_national_list["records"][0]["data"].keys())

dict_keys(['data', 'run_id', 'row_id', 'uri'])
dict_keys(['id', 'scientific name', 'common name', 'taxonomic category', 'state', 'sciencebase_item_id', 'record_processed', 'source_file_date', 'source_file_url', 'year', 'clean_scientific_name', 'historic_list', 'itis_override_id', 'sppin_key', 'taxogroupings', 'scientificname', 'taxonomicrank', 'taxonomic_authority_url', 'match_method', 'commonname', 'class_name', 'nationallist'])
